# 9. Decorators and Closures

> There’s been a number of complaints about the choice of the name “decorator” for this feature. The major one is that the name is not consistent with its use in the GoF book.1 The name decorator probably owes more to its use in the compiler area—a syntax tree is walked and annotated.
> 
> PEP 318—Decorators for Functions and Methods

## Decorators 101

A decorator is a callable that takes another function as an argument (the decorated function).

```python
@decorate
def target():
    print("running target()")
```
has the same effect as writing this:
```python
def target():
    print("running target()")

target = decorate(target)
```

In [ ]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

In [ ]:
target()

running inner()


In [3]:
target

<function __main__.deco.<locals>.inner()>

Strictly speaking, decorators are just syntactic sugar.

Three essential facts make a good summary of decorators:

+ A decorator is a function or another callable.
+ A decorator may replace the decorated function with a different one.
+ Decorators are executed immediately when a module is loaded.


In [ ]:
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('registry->', registry)
    f1()
    f2()
    f3()


running register(<function f1 at 0x7fd8da7b5f80>)
running register(<function f2 at 0x7fd8da7b60c0>)


In [5]:
main()

running main()
registry-> [<function f1 at 0x7fd8da7b5f80>, <function f2 at 0x7fd8da7b60c0>]
running f1()
running f2()
running f3()


In [6]:
registry

[<function __main__.f1()>, <function __main__.f2()>]

## Registration Decorators

Considering how decorators are commonly employed in real code the example is unusual in two ways:

+ The decorator function is defined in the same module as the decorated functions. A real decorator is usually defined in one module and applied to functions in other modules.
+ The `register` decorator returns the same function passed as an argument. In practice, most decorators define an inner function and return it.

## Variable Scope Rules

In [7]:
def f1(a):
    print(a)
    print(b)

try:
    f1(3)
except Exception as e:
    print(f"{e=}")

3
e=NameError("name 'b' is not defined")


In [8]:
b = 6
f1(3)

3
6


In [9]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 0

try: f2(3)
except Exception as e: print(f"{e=}")

3
e=UnboundLocalError("cannot access local variable 'b' where it is not associated with a value")


Note that the output starts with `3`, which proves that the `print(a)` statement was executed. But the second one, `print(b)`, never runs. When I first saw this I was surprised, thinking that `6` should be printed, because there is a global variable `b` and the assignment to the local b is made after `print(b)`.

But the fact is, when Python compiles the body of the function, it decides that `b` is a local variable because it is assigned within the function. The generated bytecode reflects this decision and will try to fetch `b` from the local scope. Later, when the call `f2(3)` is made, the body of `f2` fetches and prints the value of the local variable `a`, but when trying to fetch the value of local variable `b`, it discovers that `b` is unbound.

In [10]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
    
f3(3)

3
6


In [11]:
b

9

## Closures

In the blogosphere, closures are sometimes confused with anonymous functions. Many confuse them because of the parallel history of those features: defining functions inside functions is not so common or convenient, until you have anonymous functions. And closures only matter when you have nested functions. So a lot of people learn both concepts at the same time.

In [12]:
class Averager():
    
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)


In [13]:
avg = Averager()
avg(10)

10.0

In [14]:
avg(11)

10.5

In [15]:
avg(12)

11.0

In [16]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

In [17]:
avg = make_averager()
avg(10)

10.0

In [18]:
avg(11)

10.5

In [19]:
avg(15)

12.0

In [20]:
avg.__code__.co_varnames

('new_value', 'total')

In [21]:
avg.__code__.co_freevars

('series',)

In [22]:
avg.__closure__

(<cell at 0x7fd8da7d5f60: list object at 0x7fd8da611e00>,)

In [23]:
avg.__closure__[0].cell_contents

[10, 11, 15]

## The **nonlocal** Declaration

Our previous implementation of `make_averager` was not efficient. We stored all the values in the historical `series` and computed their `sum` every time `averager` was called. A better implementation would only store the total and the number of items so far, and compute the mean from these two numbers.

In [24]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

In [25]:
try:
    avg = make_averager()
    avg(10)
except Exception as e: print(f"{e=}")

e=UnboundLocalError("cannot access local variable 'count' where it is not associated with a value")


In [26]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

In [27]:
try:
    avg = make_averager()
    print(f"{avg(10)=}")
except Exception as e: print(f"{e=}")

avg(10)=10.0


## Implementing a Simple Decorator

The example is a decorator that clocks every invocation of the decorated function and displays the elapsed time, the arguments passed, and the result of the call.

In [28]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    
    return clocked

In [29]:
@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n<2 else n*factorial(n-1)

In [30]:
print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12306752s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000561s] factorial(1) -> 1
[0.00002214s] factorial(2) -> 2
[0.00003442s] factorial(3) -> 6
[0.00004622s] factorial(4) -> 24
[0.00005744s] factorial(5) -> 120
[0.00006293s] factorial(6) -> 720
6! = 720


In [31]:
factorial.__name__

'clocked'

The `clock` decorator implemented in Example has a few shortcomings: it does not support keyword arguments, and it masks the `__name__` and `__doc__` of the decorated function. Next example uses the `functools.wraps` decorator to copy the relevant attributes from `func` to `clocked`. Also, in this new version, keyword arguments are correctly handled.

In [32]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = [repr(arg) for arg in args]
        arg_lst.extend(f'{k}={v!r}' for k, v in kwargs.items())
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    
    return clocked

In [33]:
print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12324049s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000043s] factorial(1) -> 1
[0.00000957s] factorial(2) -> 2
[0.00001897s] factorial(3) -> 6
[0.00002431s] factorial(4) -> 24
[0.00003047s] factorial(5) -> 120
[0.00003641s] factorial(6) -> 720
6! = 720


## Decorators in the Standard Library

Python has three built-in functions that are designed to decorate methods: `property`, `classmethod`, and `staticmethod`. 

### Memoization with `functools.cache`

The `functools.cache` decorator implements memoization: an optimization technique that works by saving the results of previous invocations of an expensive function, avoiding repeat computations on previously used arguments.

In [34]:
@clock
def fibonacci(n):
    if n<2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [35]:
print(f"{fibonacci(6)=}")

[0.00000031s] fibonacci(0) -> 0
[0.00000041s] fibonacci(1) -> 1
[0.00003516s] fibonacci(2) -> 1
[0.00000025s] fibonacci(1) -> 1
[0.00000041s] fibonacci(0) -> 0
[0.00000033s] fibonacci(1) -> 1
[0.00001583s] fibonacci(2) -> 1
[0.00002903s] fibonacci(3) -> 2
[0.00007600s] fibonacci(4) -> 3
[0.00000018s] fibonacci(1) -> 1
[0.00000014s] fibonacci(0) -> 0
[0.00000021s] fibonacci(1) -> 1
[0.00000985s] fibonacci(2) -> 1
[0.00001968s] fibonacci(3) -> 2
[0.00000018s] fibonacci(0) -> 0
[0.00000023s] fibonacci(1) -> 1
[0.00000971s] fibonacci(2) -> 1
[0.00000015s] fibonacci(1) -> 1
[0.00000015s] fibonacci(0) -> 0
[0.00000018s] fibonacci(1) -> 1
[0.00000909s] fibonacci(2) -> 1
[0.00001857s] fibonacci(3) -> 2
[0.00003806s] fibonacci(4) -> 3
[0.00006686s] fibonacci(5) -> 5
[0.00015381s] fibonacci(6) -> 8
fibonacci(6)=8


In [36]:
import functools

@functools.cache
@clock
def fibonacci(n):
    if n<2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [37]:
print(f"{fibonacci(6)=}")

[0.00000027s] fibonacci(0) -> 0
[0.00000035s] fibonacci(1) -> 1
[0.00003648s] fibonacci(2) -> 1
[0.00000061s] fibonacci(3) -> 2
[0.00004930s] fibonacci(4) -> 3
[0.00000061s] fibonacci(5) -> 5
[0.00006212s] fibonacci(6) -> 8
fibonacci(6)=8


### Using `lru_cache`

The `functools.cache` decorator is actually a simple wrapper around the older `functools.lru_cache` function, which is more flexible and compatible with Python 3.8 and earlier versions.

The main advantage of `@lru_cache` is that its memory usage is bounded by the `maxsize` parameter, which has a rather conservative default value of `128`—which means the cache will hold at most 128 entries at any time.

Since Python 3.8, `lru_cache` can be applied in two ways. This is how to use it in the simplest way:
 
```python
@lru_cache
def costly_function(a, b):
    ...
```
The other way—available since Python 3.2—is to invoke it as a function, with `()`:
```python
@lru_cache()
def costly_function(a, b):
    ...
```

In both cases, the default parameters would be used.

`maxsize=128` Sets the maximum number of entries to be stored. 
 
`typed=False` Determines whether the results of different argument types are stored separately.

### Single Dispatch Generic Functions

Because we don’t have Java-style method overloading in Python, we can’t simply create variations of htmlize with different signatures for each data type we want to han‐ dle differently. A possible solution in Python would be to turn htmlize into a dispatch function, with a chain of `if/elif/...` or `match/case/...` calling specialized functions like `htmlize_str`, `htmlize_int`, etc. This is not extensible by users of our module, and is unwieldy: over time, the htmlize dispatcher would become too big, and the coupling between it and the specialized functions would be very tight.

## Parametrized Decorators

When parsing a decorator in source code, Python takes the decorated function and passes it as the first argument to the decorator function. So how do you make a deco‐ rator accept other arguments? The answer is: make a decorator factory that takes those arguments and returns a decorator, which is then applied to the function to be decorated. Confusing? Sure. 

In [38]:
registry = set()

def register(active=True):
    def decorate(func):
        print('running register'
              f'(active={active})->decorate({func})'
              )
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        
        return func
    return decorate


In [39]:
@register(active=False)
def f1():
    print('running f1()')
    
@register()
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x7fd8da7b6c00>)
running register(active=True)->decorate(<function f2 at 0x7fd8da7b6700>)


In [40]:
registry

{<function __main__.f2()>}

In [41]:
register()(f3)

running register(active=True)->decorate(<function f3 at 0x7fd8da7b6660>)


<function __main__.f3()>

In [42]:
registry

{<function __main__.f2()>, <function __main__.f3()>}

In [43]:
register(active=False)(f2)

running register(active=False)->decorate(<function f2 at 0x7fd8da7b6700>)


<function __main__.f2()>

In [44]:
registry

{<function __main__.f3()>}

## The Parametrized Clock Decorator

In [45]:
DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

In [46]:
@clock()
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

[0.12326960s] snooze(0.123) -> None
[0.12323237s] snooze(0.123) -> None
[0.12309761s] snooze(0.123) -> None


In [47]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.1233012571465224s
snooze: 0.12323595490306616s
snooze: 0.12309592799283564s


In [48]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s


### A Class-Based Clock Decorator

As a final example, a new example lists the implementation of a parameterized clock decorator implemented as a class with `__call__`.


In [49]:
DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

class clock:
    
    def __init__(self, fmt=DEFAULT_FMT):
        self.fmt = fmt
    
    def __call__(self, func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return clocked

In [50]:
@clock
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)